In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [2]:
import pandas as pd
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
import os, glob
import time

import nltk
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\josed\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def checker2(wrong_options, correct_options):
    i=0
    results = [] 
    for wrong_option in wrong_options:
#        print(wrong_option)
        
#        buenas = wrong_options[wrong_options.Nombre == wrong_option]['Nielsen'].unique()
#        print(buenas)
#        correct_options = correct_options_2[correct_options_2.Desc_Categ.isin(buenas) ]['texto_limpio']
#        print(correct_options)
        
        x = process.extract(wrong_option, correct_options, scorer=fuzz.token_sort_ratio, limit = 1)
#        print(x)
        x = pd.DataFrame(x, columns = ['CONCEPTO_BANORTE','Score2','Familia'])

        #y = process.extract(wrong_option, correct_options, scorer=fuzz.partial_ratio, limit = 6)
        #y = pd.DataFrame(y, columns = ['Similares3','Score3'])

        #x = pd.concat([x, y], axis = 1)
        x = pd.concat([x], axis = 1)

        x['CONCEPTO_BBVA'] = wrong_option
 #       x['tabla_banorte'] = tabla_banorte
 #       x['tabla_santander'] = tabla_santander
        
        results.append(x)
        i+=1
#        print(i)   
    result = pd.concat( results , sort = False)
#    result['articulo_asociado'] = tipo
    return result

In [4]:
def checker3(wrong_options, correct_options):
    i=0
    results = [] 
    for wrong_option in wrong_options:
#        print(wrong_option)
        
#        buenas = wrong_options[wrong_options.Nombre == wrong_option]['Nielsen'].unique()
#        print(buenas)
#        correct_options = correct_options_2[correct_options_2.Desc_Categ.isin(buenas) ]['texto_limpio']
#        print(correct_options)
        
        x = process.extract(wrong_option, correct_options, scorer=fuzz.token_sort_ratio, limit = 1)
#        print(x)
        x = pd.DataFrame(x, columns = ['CONCEPTO_SANTANDER','Score2','Familia'])

        #y = process.extract(wrong_option, correct_options, scorer=fuzz.partial_ratio, limit = 6)
        #y = pd.DataFrame(y, columns = ['Similares3','Score3'])

        #x = pd.concat([x, y], axis = 1)
        x = pd.concat([x], axis = 1)

        x['CONCEPTO_BBVA'] = wrong_option
 #       x['tabla_banorte'] = tabla_banorte
 #       x['tabla_santander'] = tabla_santander
        
        results.append(x)
        i+=1
#        print(i)   
    result = pd.concat( results , sort = False)
#    result['articulo_asociado'] = tipo
    return result

In [5]:
trimestres = ['1T21','2T21','2T20','3T20','4T20']

In [91]:
trimestre = '2T21'
reporte = 'Coeficiente_de_cobertura'

In [92]:
int(trimestre[-2:])

21

In [93]:
lista_bbva = (os.listdir('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/BBVA_'+reporte+'_'+trimestre))      
tablas_bbva = [s for s in lista_bbva if "table" in s]
bbva = pd.DataFrame()
for i in tablas_bbva:
#        print(i)
    globals()[f'bbva_'+ i[-5]] = pd.read_csv('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/BBVA_'+reporte+'_'+trimestre+'/'+i)
    globals()[f'bbva_'+ i[-5]].columns = globals()[f'bbva_'+ i[-5]].columns.str.replace(' 20','')
    globals()[f'bbva_'+ i[-5]] = globals()[f'bbva_'+ i[-5]].rename(columns = {globals()[f'bbva_'+ i[-5]].columns[0] : 'CONCEPTO'})
    bbva = pd.concat([bbva,globals()[f'bbva_'+ i[-5]]],axis = 0)


vec_c = []
for columna in bbva.columns:
    nueva_columna = str(columna)+'_BBVA'
    vec_c.append(nueva_columna)
bbva.columns = vec_c 
    
banorte = pd.DataFrame()    
lista_banorte = (os.listdir('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/Banorte_'+reporte+'_'+trimestre))      
tablas_banorte = [s for s in lista_banorte if "table" in s]
for i in tablas_banorte:
#        print(i)
    globals()[f'banorte_'+ i[-5]] = pd.read_csv('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/Banorte_'+reporte+'_'+trimestre+'/'+i)
    globals()[f'banorte_'+ i[-5]].columns = globals()[f'banorte_'+ i[-5]].columns.str.replace('1T','Mar')
    globals()[f'banorte_'+ i[-5]].columns = globals()[f'banorte_'+ i[-5]].columns.str.replace('2T','Jun')
    globals()[f'banorte_'+ i[-5]].columns = globals()[f'banorte_'+ i[-5]].columns.str.replace('3T','Sep')
    globals()[f'banorte_'+ i[-5]].columns = globals()[f'banorte_'+ i[-5]].columns.str.replace('4T','Dic')
    globals()[f'banorte_'+ i[-5]] = globals()[f'banorte_'+ i[-5]].rename(columns = {globals()[f'banorte_'+ i[-5]].columns[0] : 'CONCEPTO'})

    banorte = pd.concat([banorte,globals()[f'banorte_'+ i[-5]]],axis = 0)
   
vec_c = []
for columna in banorte.columns:
    nueva_columna = str(columna)+'_BANORTE'
    vec_c.append(nueva_columna)
banorte.columns = vec_c

santander = pd.DataFrame()   
lista_santander = (os.listdir('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/Santander_'+reporte+'_'+trimestre))      
tablas_santander = [s for s in lista_santander if "table" in s]
anio = int(trimestre[-2:])
for i in tablas_santander[:1]:
    globals()[f'santander_'+ i[-5]] = pd.read_csv('C:/Users/josed/Downloads/OUTPUT_BBVA_2021_3/Output/Santander_'+reporte+'_'+trimestre+'/'+i)
    if any("Mar" in s for s in globals()[f'santander_'+ i[-5]].columns):
        print('chido') 
    else:
        globals()[f'santander_'+ i[-5]].columns =globals()[f'santander_'+ i[-5]].values[0]
    #print(globals()[f'santander_'+ i[-5]])
    
    
    vec = []
    for columna in globals()[f'santander_1'].values[0]:
        nueva_columna = str(columna)+str(anio)
        if 'Mar '+str(anio) in vec:
            nueva_columna = str(columna)+str(anio-1)
        vec.append(nueva_columna)
    
    #print(vec)
    globals()[f'santander_'+ i[-5]].columns = vec
    globals()[f'santander_'+ i[-5]].columns = globals()[f'santander_'+ i[-5]].columns.str.replace(' ','')
    globals()[f'santander_'+ i[-5]] = globals()[f'santander_'+ i[-5]].rename(columns = {globals()[f'santander_'+ i[-5]].columns[0] : 'CONCEPTO'})
    santander = pd.concat([santander,globals()[f'santander_'+ i[-5]]],axis = 0)    
    
vec_c = []
for columna in santander.columns:
    nueva_columna = str(columna)+'_SANTANDER'
    vec_c.append(nueva_columna)
santander.columns = vec_c


In [94]:
banorte.head()

,CONCEPTO_BANORTE,Banco y _BANORTE,Sofomes _BANORTE,Unnamed: 3_BANORTE
0,Componentes del CCL (Millones de pesos),Importe Sin Ponderar (Promedio),Importe Ponderado (Promedio),NaN
1,ACTIVOS LÍQUIDOS COMPUTABLES,NaN,NaN,NaN
2,1 Total de Activos Líquidos Computables,No aplica,"138,335",NaN
3,SALIDAS DE EFECTIVO,NaN,NaN,NaN
4,2 Financiamiento minorista no garantizado,"441,119","28,582",NaN


In [95]:
bbva = bbva.iloc[:, :3]
bbva.columns = ['CONCEPTO_BBVA', 'BBVA_IMPORTE_SIN_PONDERAR'+trimestre , 'BBVA_IMPORTE_PONDERADO'+trimestre]
banorte = banorte.iloc[:, :3]
banorte.columns = ['CONCEPTO_BANORTE', 'BANORTE_IMPORTE_SIN_PONDERAR'+trimestre , 'BANORTE_IMPORTE_PONDERADO'+trimestre]
santander = santander.iloc[:, :3]
santander.columns = ['CONCEPTO_SANTANDER', 'SANTANDER_IMPORTE_SIN_PONDERAR'+trimestre , 'SANTANDER_IMPORTE_PONDERADO'+trimestre]
#banorte = banorte.drop(columns = 'Unnamed: 9_BANORTE')
#santander = santander.drop(columns = 'nan19_SANTANDER')

In [98]:
final = pd.DataFrame()
finals = []

result = checker2(bbva.CONCEPTO_BBVA, banorte.CONCEPTO_BANORTE)


finals.append(result)
final = pd.concat(finals)

final_2 = pd.DataFrame()
finals_2 = []

result_2 = checker3(bbva.CONCEPTO_BBVA, santander.CONCEPTO_SANTANDER)


finals_2.append(result_2)
final_2 = pd.concat(finals_2)

In [99]:
conceptos = final[final.Score2>60]
bbva_2 = bbva.merge(conceptos[['CONCEPTO_BBVA','CONCEPTO_BANORTE']],how = 'left')
balance = bbva_2.merge(banorte , how = 'left')
balance  = balance.drop_duplicates()

In [100]:
conceptos2 = final_2[final_2.Score2>60]
balance_2 = balance.merge(conceptos2[['CONCEPTO_BBVA','CONCEPTO_SANTANDER']],how = 'left')
balance_2 = balance_2.merge(santander , how = 'left')
balance_2  = balance_2.drop_duplicates()

In [101]:
balance_2

,CONCEPTO_BBVA,BBVA_IMPORTE_SIN_PONDERAR2T21,BBVA_IMPORTE_PONDERADO2T21,CONCEPTO_BANORTE,BANORTE_IMPORTE_SIN_PONDERAR2T21,BANORTE_IMPORTE_PONDERADO2T21,CONCEPTO_SANTANDER,SANTANDER_IMPORTE_SIN_PONDERAR2T21,SANTANDER_IMPORTE_PONDERADO2T21
0,Millones de pesos,Importe sin Ponderar (promedio),Importe Ponderado (promedio),Componentes del CCL (Millones de pesos),Importe Sin Ponderar (Promedio),Importe Ponderado (Promedio),NaN,NaN,Importe ajustado
1,ACTIVOS LÍQUIDOS COMPUTABLES,NaN,NaN,ACTIVOS LÍQUIDOS COMPUTABLES,NaN,NaN,Activos Líquidos Computables,NaN,NaN
2,1 Total de Activos Líquidos Computables,No Aplica,"433,973",1 Total de Activos Líquidos Computables,No aplica,"138,335",1 Total de activos líquidos computables,No aplica,"225,575"
3,SALIDAS DE EFECTIVO,NaN,NaN,SALIDAS DE EFECTIVO,NaN,NaN,Salidas de Efectivo,NaN,NaN
4,2 Financiamiento Minorista No Garantizado,"786,248","48,508",2 Financiamiento minorista no garantizado,"441,119","28,582",2 Financiamiento minorista no garantizado,"277,487","15,227"
5,3 Financiamiento Estable,"602,338","30,117",3 Financiamiento estable,"310,591","15,530",3 Financiamiento estable,"250,438","12,522"
6,4 Financiamiento menos Estable,"183,910","18,391",4 Financiamiento menos estable,"130,528","13,053",4 Financiamiento menos estable,"27,048","2,705"
7,5 Financiamiento Mayorista No Garantizado,"479,208","152,221",5 Financiamiento mayorista no garantizado,"278,317","75,624",5 Financiamiento mayorista no garantizado,"434,690","161,503"
8,6 Depósitos Operacionales,"292,636","70,673",6 Depósitos operacionales,"242,107","55,134",6 Depósitos operacionales,"234,579","53,968"
9,7 Depósitos No Operacionales,"182,376","77,351",7 Depósitos no operacionales,"35,071","19,351",7 Depósitos no operacionales,"176,324","83,748"


In [102]:
balance_2.to_csv('C:/Users/josed/Downloads/COEFICIENTES_COBERTURA/COEFICIENTES_COBERTURA_'+trimestre+'.csv')

In [103]:
balance_2

,CONCEPTO_BBVA,BBVA_IMPORTE_SIN_PONDERAR2T21,BBVA_IMPORTE_PONDERADO2T21,CONCEPTO_BANORTE,BANORTE_IMPORTE_SIN_PONDERAR2T21,BANORTE_IMPORTE_PONDERADO2T21,CONCEPTO_SANTANDER,SANTANDER_IMPORTE_SIN_PONDERAR2T21,SANTANDER_IMPORTE_PONDERADO2T21
0,Millones de pesos,Importe sin Ponderar (promedio),Importe Ponderado (promedio),Componentes del CCL (Millones de pesos),Importe Sin Ponderar (Promedio),Importe Ponderado (Promedio),NaN,NaN,Importe ajustado
1,ACTIVOS LÍQUIDOS COMPUTABLES,NaN,NaN,ACTIVOS LÍQUIDOS COMPUTABLES,NaN,NaN,Activos Líquidos Computables,NaN,NaN
2,1 Total de Activos Líquidos Computables,No Aplica,"433,973",1 Total de Activos Líquidos Computables,No aplica,"138,335",1 Total de activos líquidos computables,No aplica,"225,575"
3,SALIDAS DE EFECTIVO,NaN,NaN,SALIDAS DE EFECTIVO,NaN,NaN,Salidas de Efectivo,NaN,NaN
4,2 Financiamiento Minorista No Garantizado,"786,248","48,508",2 Financiamiento minorista no garantizado,"441,119","28,582",2 Financiamiento minorista no garantizado,"277,487","15,227"
5,3 Financiamiento Estable,"602,338","30,117",3 Financiamiento estable,"310,591","15,530",3 Financiamiento estable,"250,438","12,522"
6,4 Financiamiento menos Estable,"183,910","18,391",4 Financiamiento menos estable,"130,528","13,053",4 Financiamiento menos estable,"27,048","2,705"
7,5 Financiamiento Mayorista No Garantizado,"479,208","152,221",5 Financiamiento mayorista no garantizado,"278,317","75,624",5 Financiamiento mayorista no garantizado,"434,690","161,503"
8,6 Depósitos Operacionales,"292,636","70,673",6 Depósitos operacionales,"242,107","55,134",6 Depósitos operacionales,"234,579","53,968"
9,7 Depósitos No Operacionales,"182,376","77,351",7 Depósitos no operacionales,"35,071","19,351",7 Depósitos no operacionales,"176,324","83,748"


In [104]:
b1 = pd.read_csv('C:/Users/josed/Downloads/COEFICIENTES_COBERTURA/COEFICIENTES_COBERTURA_2T20.csv').drop(columns = 'Unnamed: 0')
b2 = pd.read_csv('C:/Users/josed/Downloads/COEFICIENTES_COBERTURA/COEFICIENTES_COBERTURA_3T20.csv').drop(columns = 'Unnamed: 0')
b3 = pd.read_csv('C:/Users/josed/Downloads/COEFICIENTES_COBERTURA/COEFICIENTES_COBERTURA_4T20.csv').drop(columns = 'Unnamed: 0')
b4 = pd.read_csv('C:/Users/josed/Downloads/COEFICIENTES_COBERTURA/COEFICIENTES_COBERTURA_1T21.csv').drop(columns = 'Unnamed: 0')
b5 = pd.read_csv('C:/Users/josed/Downloads/COEFICIENTES_COBERTURA/COEFICIENTES_COBERTURA_2T21.csv').drop(columns = 'Unnamed: 0')
b1 = b1.merge(b2[list(set(b2.columns )- set (b1.columns))+['CONCEPTO_BBVA']],how = 'left')
b1 = b1.merge(b3[list(set(b3.columns )- set (b1.columns))+['CONCEPTO_BBVA']],how = 'left')
b1 = b1.merge(b4[list(set(b4.columns )- set (b1.columns))+['CONCEPTO_BBVA']],how = 'left')
b1 = b1.merge(b5[list(set(b5.columns )- set (b1.columns))+['CONCEPTO_BBVA']],how = 'left')

In [106]:
b1.to_csv('COEFICIENTES_COBERTURA_join_completo.csv')